In [104]:
import tensorflow as tf
from tensorboard import version;
print("Tensorboard version: ", version.VERSION)
#load_ext tensorboard
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras import backend as K

Tensorboard version:  2.0.2


In [101]:
import numpy as np
import json
from time import time
import random

In [2]:
print(tf.__version__)

2.0.0


In [40]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

In [75]:
def read_train_data(filename):
    with open(filename) as f:
        train = json.load(f)
    return train

In [76]:
train_data = read_train_data("data/PaperQA-train.json")

In [77]:
def collate_data(data):
    cleaned_data = []
    for d in data['data']:
        try:
            title = d["Paper"]["Title"].lower().strip().split()
            abst = d["Paper"]["Abstract"].lower().strip().split()
            cl = " ".join(title + abst)
            cleaned_data.append(cl)
        except Exception as ex:
            continue
    return cleaned_data

In [78]:
cleaned_data = collate_data(train_data)

In [79]:
print(train_data['data'][0]["Paper"]["Title"])
print(train_data['data'][0]["Paper"]["Abstract"])

Object categorization based on a supervised mean shift algorithm
In this work, we present a C&#43;&#43; implementation of object categorization with the bag-of-word (BoW) framework. Unlike typical BoW models which consider the whole area of an image as the region of interest (ROI) for visual codebook generation, our implementation only considers the regions of target objects as ROIs and the unrelated backgrounds will be excluded for generating codebook. This is achieved by a supervised mean shift algorithm. Our work is on the benchmark SIVAL dataset and utilizes a Maximum Margin Supervised Topic Model for classification. The final performance of our work is quite encouraging.


In [80]:
print("Length of train set(#papers): ", len(cleaned_data))
print(cleaned_data[1])

Length of train set(#papers):  1424
sequence to sequence learning with neural networks deep neural networks (dnns) are powerful models that have achieved excellent performance on difficult learning tasks. although dnns work well whenever large labeled training sets are available, they cannot be used to map sequences to sequences. in this paper, we present a general end-to-end approach to sequence learning that makes minimal assumptions on the sequence structure. our method uses a multilayered long short-term memory (lstm) to map the input sequence to a vector of a fixed dimensionality, and then another deep lstm to decode the target sequence from the vector. our main result is that on an english to french translation task from the wmt-14 dataset, the translations produced by the lstm achieve a bleu score of 34.8 on the entire test set, where the lstm's bleu score was penalized on out-of-vocabulary words. additionally, the lstm did not have difficulty on long sentences. for comparison, 

In [81]:
full_text = " ".join(cleaned_data)
vocab = sorted(set(full_text))

In [82]:
print("Length of vocab: ", len(vocab))

Length of vocab:  69


In [83]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [84]:
avg_len = 0
for d in cleaned_data:
    avg_len += len(d)
print("Average length of title+abstract from 1424 papers: ", avg_len/len(cleaned_data))

Average length of title+abstract from 1424 papers:  1052.7991573033707


In [85]:
# Construct sequences
seq_length = 200
sequences = list()

for d in cleaned_data:
    for i in range(seq_length, len(d)):
        seq = d[i-seq_length: i+1]
        sequences.append(seq)

print("Total sequences (of len: %d) are: %d"%(seq_length, len(sequences)))

Total sequences (of len: 200) are: 1216481


In [86]:
# Encode sequences
en_sequences = list()
for line in sequences:
    # encode via integer
    encoded_seq = [char2idx[char] for char in line]
    en_sequences.append(encoded_seq)

In [88]:
def split_input_target(sequences):
    s = np.array(sequences)
    X, y = s[:,:-1], s[:, -1]
    
    # reduce size
#     X = X[:50000]
#     y = y[:50000]
    return X, y

X, y = split_input_target(en_sequences)

In [114]:
train_dataset = tf.data.Dataset.from_tensor_slices((X[:50000], y[:50000]))

val_dataset = tf.data.Dataset.from_tensor_slices((X[60000:70000], y[60000:70000]))

In [115]:
test_dataset = tf.data.Dataset.from_tensor_slices((X[80000:90000], y[80000:90000]))

In [50]:
vocab_size = len(vocab)

In [ ]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size)

In [67]:
# bidir_layer = Bidirectional(LSTM(1000))
# pen_dense_layer = Dense(69, activation='softmax')
# final_dense_layer = Dense(100, activation='softmax')

# model = Sequential()
# model.add(bidir_layer)
# model.add(final_dense_layer)
# # print(model.summary())

In [68]:
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [56]:
model.fit??

In [66]:
# y

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [69]:
# model.fit(X, y, epochs=10, verbose=1)

Train on 50000 samples
Epoch 1/10
   32/50000 [..............................] - ETA: 51:36

InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[32,100] labels_size=[32,69]
	 [[node loss/output_1_loss/softmax_cross_entropy_with_logits (defined at /home/singh_shruti/workspace/tftest/lib64/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_12375]

Function call stack:
distributed_function


In [29]:
tf.keras.layers.Bidirectional??

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 100

# Number of RNN units
rnn_units = 1024

In [ ]:
# class MultiLayerRNN:
    
#     def __init__(self, teacher_forcing=False):
#         # number of units in 1st and 2nd recurrent layer, and the next dense layer
#         self.num_units_l1 = hp.HParam('num_units_l1', hp.Discrete([64, 128, 256]))
#         self.num_units_l2 = hp.HParam('num_units_l2', hp.Discrete([64, 128, 256]))
#         #self.dropout = hp.HParam('dropout', hp.Discrete([0.3, 0.4]))

#         self.optimizer = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
        
#         self.hparams = {self.optimizer: self.optimizer, self.num_units_l1: self.num_units_l1, 
#                         self.num_units_l2: self.num_units_l2}
#                         #self.dropout: self.dropout}
        
#         self.teacher_forcing = teacher_forcing
        
#         self.model = None
        
#         METRIC_ACCURACY = 'accuracy'
        
#         self.init_timestamp = int(time())
        
#         print("MODEL INIT TIMESTAMP: ", str(self.init_timestamp))
        
#         self.log_dir = "./tf_logs/char_emb_" + str(self.init_timestamp) +"/"
#         with tf.summary.create_file_writer(self.log_dir).as_default():
#             #self.dropout
#             hp.hparams_config(hparams=[self.optimizer, self.num_units_l1, self.num_units_l2], 
#                               metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],)
        
#         return
    
#     def loss_function(self, y_true, y_pred):
#         return tf.keras.losses.binary_crossentropy(y_true, y_pred)
    
#     def generate_model(self, params):
        
#         self.model = tf.keras.Sequential()
#         self.model.add(tf.keras.layers.Embedding(input_dim=encoder.vocab_size, output_dim=64))
#         self.model.add(tf.keras.layers.RNN([CustomRNNCell(params[self.num_units_l1]), 
#                                             CustomRNNCell(params[self.num_units_l2])]))
#         self.model.add(tf.keras.layers.Dropout(params[self.dropout]))
#         self.model.add(tf.keras.layers.Dense(params[self.num_units_l3], activation='relu'))
#         self.model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
        
#         print(self.model.summary())
#         return self.model
    
#     def get_model(self):
#         return self.model
    
#     def save_model(self):
#         cp = time()
#         model.save_weights(self.logdir + '/saved_models/model_' + cp, save_format='tf')
#         return
    
#     def compile_model(self, loss_function, optimizer):
#         self.model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])
#         return self.model
    
#     def train_model(self, hparams, train_data, cross_validation_data, run_index):
#         self.generate_model(hparams)
#         self.compile_model(self.loss_function, hparams[self.optimizer])
#         #self.compile_model(self.loss_function, self.optimizer)
        
#         callbacks = [
#             # Early stopping
#             tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4),
#             # logging train and validation accuracy each epoch
#             tf.keras.callbacks.TensorBoard(log_dir=self.log_dir + run_index),
#             # gradient sum callback
#             gradient_cb,
#             # checkpoint
#             #tf.keras.callbacks.ModelCheckpoint(filepath=, save_weights_only=True, save_best_only=True)
#         ]
        
#         self.model.fit(train_data, epochs=10, validation_data=cross_validation_data, callbacks=callbacks, 
#                        verbose=1)
#         _, accuracy = self.model.evaluate(cross_validation_data)
        
#         return accuracy
    
#     def run(self, run_dir, hparams, train_data, cross_validation_data):
#         K.clear_session()
#         run_index = run_dir.split("-")[1]
#         with tf.summary.create_file_writer(run_dir).as_default():
#             # record the values used in this trial
#             hp.hparams(hparams)
#             acc = self.train_model(hparams, train_dataset, validation_dataset, run_index)
#             tf.summary.scalar('accuracy', acc, step=int(run_index))
#         return acc
    
#     def random_search(self, train, cross_val, seed):
#         rng = random.Random(seed)
#         total_points_explored = 1
        
#         acc_params = []
        
#         for session_index in range(total_points_explored):
#             hparams = {h: h.domain.sample_uniform(rng) for h in self.hparams}
#             run_name = "run-%d" % session_index
#             print('--- Starting trial: %s' % run_name)
#             print({h.name: hparams[h] for h in hparams})
#             acc = self.run(self.log_dir + "tune/" + run_name, hparams, train, cross_val)
#             session_index += 1
#             acc_params.append((acc, hparams))
        
#         return total_points_explored, acc_params
    
#     def setup_model(self, params):
#         K.clear_session()
#         self.generate_model(params)
#         self.compile_model(self.loss_function, params[self.optimizer])
#         return
    
#     def test_params(self, hpa):
#         K.clear_session()
#         params = {h: hpa[h.name] for h in self.hparams}
#         self.generate_model(params)
#         self.compile_model(self.loss_function, params[self.optimizer])
#         self.run(self.log_dir + "tune/testhpgrad-1", params, train_dataset, validation_dataset)
#         return
    
#     def eval_test(self, test):
#         _, acc = self.model.evaluate(test)
#         print("Accuracy on test set: ", acc)
#         return


In [116]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

# fills a buffer with buffer_size elements, then randomly samples elements from this buffer, replacing the selected elements with new elements. For perfect shuffling, a buffer size greater than or equal to the full size of the dataset is required.
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
# output_shapes returns the shape of each component of an element of this dataset.
train_dataset = train_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_dataset))

val_dataset = val_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(val_dataset))

test_dataset = test_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_dataset))

In [117]:
class MultiLayerLSTM:
    
    def __init__(self, teacher_forcing=False):
        # number of units in 1st and 2nd LSTM layer, and the next dense layer
        self.num_units_l1 = hp.HParam('num_units_l1', hp.Discrete([64]))
        self.num_units_l2 = hp.HParam('num_units_l2', hp.Discrete([64]))
        self.num_units_l3 = hp.HParam('num_units_l3', hp.Discrete([100]))
        self.optimizer = hp.HParam('optimizer', hp.Discrete(['adam']))
        
        self.hparams = {self.optimizer: self.optimizer, self.num_units_l1: self.num_units_l1, self.num_units_l2: self.num_units_l2, self.num_units_l3: self.num_units_l3}
        
        self.teacher_forcing = teacher_forcing
        
        self.model = None
        
        METRIC_ACCURACY = 'accuracy'
        
        self.timestamp = int(time())
        print("MODEL INIT TIME: ", str(self.timestamp))
        self.log_dir = "./tf_logs/char_emb_" + str(self.timestamp) +"/"
        with tf.summary.create_file_writer(self.log_dir).as_default():
            hp.hparams_config(hparams=[self.optimizer, self.num_units_l1, self.num_units_l2, self.num_units_l3], metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],)
        
        return
    
    def loss_function(self, y_true, y_pred):
        return tf.keras.losses.binary_crossentropy(y_true, y_pred)
    
    def generate_model(self, params, single_layer=True):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=64))
        self.model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(params[self.num_units_l1], return_sequences=True)))
        self.model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(params[self.num_units_l2])))
        self.model.add(tf.keras.layers.Dense(params[self.num_units_l3], activation='relu'))
        self.model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
        print(self.model.summary())
        return self.model
    
    def get_model(self):
        return self.model
    
    def save_model(self):
        cp = int(time())
        model.save_weights(self.logdir + '/saved_models/model_' + cp, save_format='tf')
        return
    
    def compile_model(self, loss_function, optimizer):
        self.model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])
        return self.model
    
    def train_model(self, hparams, train_data, cross_validation_data, run_index):
        self.generate_model(hparams)
        self.compile_model(self.loss_function, hparams[self.optimizer])
        
        callbacks = [
            # Early stopping
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4),
            tf.keras.callbacks.TensorBoard(log_dir=self.log_dir + run_index),
        ]
        
        self.model.fit(train_data, epochs=3, validation_data=cross_validation_data, callbacks=callbacks,verbose=1)
        _, accuracy = self.model.evaluate(cross_validation_data)
        
        return accuracy
    
    def run(self, run_dir, hparams, train_data, cross_validation_data):
        K.clear_session()
        run_index = run_dir.split("-")[1]
        with tf.summary.create_file_writer(run_dir).as_default():
            # record the values used in this trial
            hp.hparams(hparams)
            acc = self.train_model(hparams, train_dataset, cross_validation_data, run_index)
            tf.summary.scalar('accuracy', acc, step=int(run_index))
        return acc
    
    def random_search(self, train, cross_val, seed):
        rng = random.Random(seed)
        total_points_explored = 1
        
        acc_params = []
        
        for session_index in range(total_points_explored):
            hparams = {h: h.domain.sample_uniform(rng) for h in self.hparams}
            run_name = "run-%d" % session_index
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            acc = self.run(self.log_dir + "tune/" + run_name, hparams, train, cross_val)
            session_index += 1
            acc_params.append((acc, hparams))

        #Todo: call setup here only.
        
        return total_points_explored, acc_params
    
    def setup_model(self, hparams, single_layer=True):
        self.generate_model(hparams, single_layer)
        self.compile_model(self.loss_function, hparams[self.optimizer])
        return self.model
    
    def eval_test(self, test):
        _, acc = self.model.evaluate(test)
        print("Acc on tst set: ")
        print(acc)
        return

In [ ]:
m = MultiLayerLSTM()
points_explored, acc_params = m.random_search(train_dataset, val_dataset, 42)
opt_params = sorted(acc_params,key=lambda x: x[0], reverse=True)[0][1]
m.setup_model(opt_params)
m.eval_test(test_dataset)

MODEL INIT TIME:  1581394047
--- Starting trial: run-0
{'optimizer': 'adam', 'num_units_l1': 64, 'num_units_l2': 64, 'num_units_l3': 100}
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          4416      
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 100)               12900     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 182,281
Trainable params: 182,281
Non-trainable params: 0
____________________________________________